In [ ]:
import pandas as pd # type: ignore
from scipy.stats import ttest_ind # type: ignore
from sklearn.feature_extraction.text import CountVectorizer # type: ignore
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # type: ignore
from textblob import TextBlob # type: ignore
from wordcloud import WordCloud # type: ignore
import glob # type: ignore
import os # type: ignore
import swifter

In [ ]:
reviews_yelp = pd.read_parquet(r'Yelp\review.parquet')

In [ ]:
business = pd.read_csv('business_fl.csv', usecols= ['business_id'])

In [ ]:
# Filtrar reviews_yelp para que solo contenga las filas con business_id presentes en business_id
filtered_reviews_yelp = reviews_yelp[reviews_yelp['business_id'].isin(business['business_id'])]

In [ ]:
reviews_yelp = filtered_reviews_yelp.copy()

In [ ]:
reviews_yelp.info()

In [ ]:
# Extraer el año
reviews_yelp['date'] = pd.to_datetime(reviews_yelp['date']).dt.strftime('%Y-%m-%d')
reviews_yelp['date'] = pd.to_datetime(reviews_yelp['date'])  # Mantiene datetime sin la hora


# Definir el rango de fechas permitido
end_date = pd.Timestamp("2022-01-19")
start_date = end_date - pd.DateOffset(years=5)  # Calcula la fecha mínima automáticamente

# Filtrar solo los registros dentro del rango de los últimos 5 años
reviews_yelp = reviews_yelp[(reviews_yelp["date"] >= start_date) & (reviews_yelp["date"] <= end_date)]

In [ ]:
# Eliminar las columnas "useful", "funny", "cool", "user_id" porque no es util para los análisis
reviews_yelp = reviews_yelp.drop(columns=["useful", "funny", "cool", "user_id"])

In [ ]:
# Definir el tamaño del chunk
chunk_size = 50000  # Procesar en bloques de 50000 reviews

# Función de análisis de sentimiento
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    vader_score = analyzer.polarity_scores(text)['compound']
    textblob_score = TextBlob(text).sentiment.polarity
    return pd.Series([vader_score, textblob_score])

# Iterar sobre los datos en chunks
for start in range(0, len(reviews_yelp), chunk_size):
    end = min(start + chunk_size, len(reviews_yelp))
    chunk = reviews_yelp.iloc[start:end].copy()  # Copia para evitar modificaciones en el DataFrame original

    # Aplicar análisis de sentimiento con swifter
    chunk[['vader_score', 'textblob_score']] = chunk['text'].swifter.apply(lambda x: analyze_sentiment(str(x))).apply(pd.Series)

    # 🔹 Guardar cada chunk en un archivo Parquet para no perder datos
    chunk.to_parquet(f"sentiment_chunk_yelp_{start}.parquet")

    print(f"✅ Procesado y guardado chunk {start}-{end}")

In [ ]:
# Buscar todos los archivos guardados
chunk_files = glob.glob("sentiment_chunk_yelp_*.parquet")

# Cargar y combinar todos los chunks
reviews_yelp = pd.concat([pd.read_parquet(f) for f in chunk_files], ignore_index=True)

# Verificar que todo esté unido
print(f"Dataset final: {reviews_yelp.shape[0]} filas, {reviews_yelp.shape[1]} columnas")


In [ ]:
reviews_yelp['has_text'] = reviews_yelp['text'].notnull().astype(int)

In [ ]:
reviews_yelp = reviews_yelp.drop(columns=['text','review_id'])
reviews_yelp = reviews_yelp.rename(columns={'stars': 'rating'})

In [ ]:
# Definir el orden lógico de las columnas
column_order = ['business_id', 'rating', 'has_text', 'vader_score', 'textblob_score','date']

# Aplicar el orden de columnas asegurando que no haya errores por columnas faltantes
reviews_yelp = reviews_yelp[column_order]

# Reiniciar el índice después de todas las transformaciones
reviews_yelp.reset_index(drop=True, inplace=True)

In [ ]:
reviews_yelp.to_csv('reviews_yelp.csv', index=False)